In [1]:
from main import * 

data_dir ='data' # Change to the path to your directory
train_images_dir = os.path.join(data_dir, 'Training')
test_images_dir = os.path.join(data_dir, 'Testing')
print(f"Data directory: {data_dir}")
print(f"Training directory: {train_images_dir}")
print(f"Testing directory: {test_images_dir}")

2024-09-26 10:06:15.129316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 10:06:15.143654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 10:06:15.157987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 10:06:15.162394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 10:06:15.175909: I tensorflow/core/platform/cpu_feature_guar

Data directory: data
Training directory: data/Training
Testing directory: data/Testing


In [2]:
# Load the training and test data
try:
    train_data, test_data = get_images(train_images_dir, test_images_dir)
    print("Data loading completed successfully.")
    print(f"Number of training samples: {train_data.samples}")
    print(f"Number of test samples: {test_data.samples}")
    print(f"Class names: {train_data.class_indices}")
except Exception as e:
    print(f"Error loading data: {e}")
    raise

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Data loading completed successfully.
Number of training samples: 5712
Number of test samples: 1311
Class names: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [4]:
X_train, y_train, X_val, y_val = train_val_split(train_data)
X_test, y_test = test_splits(test_data)
class_names_train = list(train_data.class_indices.keys())
class_names_test = list(test_data.class_indices.keys())

Number of batches in the training data: 179
Batch size of a single batch 32
Number of samples in the training dataset 5712

Number of training data batches with val split of 0.2: 144
Number of validation data batches: 35

Shape of image training set: (4608, 30, 30, 1)
Shape of image validation set: (1104, 30, 30, 1)

Shape of label training set: (4608, 4)
Shape of label validation set: (1104, 4)
Number of batches in the test data: 41
Batch size of a single batch 32
Number of samples in the test dataset 1311

Shape of image test set: (1311, 30, 30, 1)

Shape of label test set: (1311, 4)


In [10]:
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
import keras

activation = "relu"
    
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(128 ,128, 3))
base_model.trainable = False


model = Sequential()
model.add(Input(shape = (30, 30, 1)))
model.add(layers.Lambda(lambda x: tf.image.resize(x, (128, 128)))) 
model.add(layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x)))
model.add(layers.Lambda(lambda x: preprocess_input(x)))
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(4, activation = "softmax"))

model.compile(optimizer = keras.optimizers.Adam(learning_rate= 0.001), 
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=3,  restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=200, validation_data = (X_val, y_val), batch_size=32, callbacks=[early_stopping])


Epoch 1/200
143/144 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3185 - loss: 1.4916

W0000 00:00:1727338271.743631    4455 assert_op.cc:38] Ignoring Assert operator sequential_5_1/lambda_16_1/grayscale_to_rgb/assert_equal_1/Assert/Assert
W0000 00:00:1727338271.744841    4455 assert_op.cc:38] Ignoring Assert operator sequential_5_1/lambda_16_1/grayscale_to_rgb/assert_greater_equal/Assert/Assert
W0000 00:00:1727338273.855314    4454 assert_op.cc:38] Ignoring Assert operator sequential_5_1/lambda_16_1/grayscale_to_rgb/assert_equal_1/Assert/Assert
W0000 00:00:1727338273.855431    4454 assert_op.cc:38] Ignoring Assert operator sequential_5_1/lambda_16_1/grayscale_to_rgb/assert_greater_equal/Assert/Assert


144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - accuracy: 0.3193 - loss: 1.4897 - val_accuracy: 0.3732 - val_loss: 1.2741
Epoch 2/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4721 - loss: 1.2053 - val_accuracy: 0.4728 - val_loss: 1.1620
Epoch 3/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.5315 - loss: 1.1118 - val_accuracy: 0.5118 - val_loss: 1.1108
Epoch 4/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.5637 - loss: 1.0525 - val_accuracy: 0.5399 - val_loss: 1.0785
Epoch 5/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5840 - loss: 1.0093 - val_accuracy: 0.5462 - val_loss: 1.0556
Epoch 6/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6017 - loss: 0.9756 - val_accuracy: 0.5571 - val_loss: 1.0379
Epoch 7/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6142 - loss: 0.9481 - val_accuracy: 0.5652 - val_loss: 1.0236
Epoch 8/200
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.6223 - loss: 0.9251 - val_accurac